In [1]:
## Parameters specific to where your folders are and your data
parameter_file = '../parameters/params.yaml'
import yaml
import sys

with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
anno_gff=doc['annotation']

import os
os.chdir(doc["results"])

from os.path import join
import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm

from os.path import basename
##mpl.use('Agg')
#mpl.style.use('ggplot')
#mpl.style.use('fivethirtyeight')
from Homer import *
from tss.data import annotation
#import annotation
print('Number of tissues: ',len(tissues))

/home/isshamie/software/anaconda2/envs/TSS/lib/python3.7/site-packages/ipykernel_launcher.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  import sys


params {'EMAIL': 'isshamie@ucsd.edu', 'parameters': 'parameters/params.yaml'}
Number of tissues:  13


In [2]:
tss_annotation = join("genome", "picr", "gcf", "start_site_mRNA_final.tsv")
mRNA_peak_file = join("genome", "picr", "gcf", "mRNA.peak") #doc["mRNA_peak_file"]


In [3]:
outdir = join("tss_annotation_peaks", "picr","gcf")
if not os.path.exists(outdir):
    os.makedirs(outdir)


In [4]:
anno_f = 'tss_annotation/picr/gcf/gene_df_01.p'
merged_f = "merged/samples.merge"

In [5]:
anno_df = pickle.load(open(anno_f,"rb"))
anno_df.head()

,peaks,closest_to_X_numberOfPeaks,bin_-1000_100,sameStrand_bin_-1000_100,samples,minDistance,minDistancePeak,maxValue,maxPeakId,maxSample,Chr,Start,End,Strand,gene,transcript,maxSamples,hasGene,Number of SS
Bmp4,"[1394, 3038, 17698, 17726, 21888, 24414, 35067...",129,"[124234, 163408, 308710, 348823]","[308710, 348823]","[MiscOrgans_START1_and_MiscOrgans_startinput1,...",26.5,308710,11864.6,308710,NaN,NW_020822366.1,724583,724584,-,Bmp4,XM_027386528.1,"[MiscOrgans_START1_and_MiscOrgans_startinput1,...",True,2
Cdkn3,"[17679, 17707, 68904, 83014, 89129, 99005, 108...",36,"[138244, 237253]",[237253],"[MiscOrgans_START1_and_MiscOrgans_startinput1,...",74.5,237253,15.3,237253,NaN,NW_020822366.1,1085509,1085510,+,Cdkn3,XM_027386531.1,"[MiscOrgans_START1_and_MiscOrgans_startinput1,...",True,1
Cnih1,"[14157, 68988, 69200, 116089, 132667, 167473, ...",17,"[132667, 298307]",[298307],"[MiscOrgans_START1_and_MiscOrgans_startinput1,...",87.5,298307,57.2,298307,NaN,NW_020822366.1,1114325,1114326,-,Cnih1,XM_027386534.1,"[MiscOrgans_START1_and_MiscOrgans_startinput1,...",True,1
Gmfb,"[34817, 35013, 35070, 86845, 97343, 120275, 12...",16,"[35013, 161643]",[],[],inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],False,0
Cgrrf1,"[5407, 35121, 224603, 293332, 306506, 338794, ...",10,"[306506, 348288]",[348288],"[MiscOrgans_START1_and_MiscOrgans_startinput1,...",68.0,348288,144.9,348288,NaN,NW_020822366.1,1159951,1159952,+,Cgrrf1,XM_027386536.1,"[MiscOrgans_START1_and_MiscOrgans_startinput1,...",True,1


In [6]:
output_f = join(outdir, "all_peaks_gene_df_01.tsv")
annotation.retrieve_all_peaks_from_anno(anno_f, merged_f, output_f)

/home/isshamie/software/anaconda2/envs/TSS/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3242: DtypeWarning: Columns (8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [7]:
output_f = join(outdir,"all_peaks_max_gene_df_01.tsv")
annotation.retrieve_all_peaks_from_anno(anno_f, merged_f, output_f,is_max=True)

In [8]:
for sample_name in glob.glob("f04_peaks/*"):
    # Merged sample file
    output_f = os.path.join(outdir,"merged_" + os.path.basename(sample_name) + '.tsv')
    annotation.retrieve_sample_peaks_from_anno(anno_f, merged_f, output_f,
                                        sample_name, sample_f=None,
                                        col_name=None,
                                        use_sample_peaks=False)
    # Original sample file
    output_f = os.path.join(outdir,"sample_" + os.path.basename(sample_name) + '.tsv')
    annotation.retrieve_sample_peaks_from_anno(anno_f, merged_f, output_f,
                                        sample_name, sample_f=sample_name,
                                        col_name=None,
                                        use_sample_peaks=True)    



In [9]:
anno_f = 'tss_annotation/picr/gcf/txn_df_02.p'
merged_f = "merged/samples.merge"
output_f = join(outdir,"all_peaks_txn_df_02.tsv")
annotation.retrieve_all_peaks_from_anno(anno_f, merged_f, output_f)

## Add onto the all_peaks_txn_df_02 the information about the maximum peak

In [6]:
merged_f = join(outdir,"all_peaks_txn_df_02.tsv")
expr_peaks_f = "merged/samples.merge.peaksexpression.log10"
out_f = join(outdir,"all_peaks_txn_df_02_maxinfo.tsv")
annotation.wrap_add_max_info(merged_f, expr_peaks_f,out_f, in_results=True)

30759it [01:51, 276.34it/s]


Results/f04_peaks/BMDM1hKLA_GROCap1_and_BMDM1hKLA_GRO1
Results/f04_peaks/BMDMwt_GROCap1_and_BMDMwt_GRO1
Results/f04_peaks/Brain_GROCap1_and_Brain_GRO1
Results/f04_peaks/Brain_START1_and_Brain_startinput1
Results/f04_peaks/Brain_START1_and_Brain_startinput2
Results/f04_peaks/Brain_START2_and_Brain_startinput1
Results/f04_peaks/Brain_START2_and_Brain_startinput2
Results/f04_peaks/Brain_START3_and_Brain_startinput1
Results/f04_peaks/Brain_START3_and_Brain_startinput2
Results/f04_peaks/CHO_GROCap1_and_CHO_GRO1
Results/f04_peaks/CHO_GROCap1_and_CHO_GRO2
Results/f04_peaks/CHO_GROCap2_and_CHO_GRO1
Results/f04_peaks/CHO_GROCap2_and_CHO_GRO2
Results/f04_peaks/CHO_START1_and_CHO_startinput1
Results/f04_peaks/FemaleReproductive_START1_and_FemaleReproductive_startinput1
Results/f04_peaks/FemaleReproductive_START2_and_FemaleReproductive_startinput1
Results/f04_peaks/Heart_START1_and_Heart_startinput1
Results/f04_peaks/Heart_START2_and_Heart_startinput1
Results/f04_peaks/Kidney_GROCap1_and_Kidney_GR

,Chr,Start,End,Strand,Stat,Parent files,Total subpeaks,Results/f04_peaks/MiscOrgans_START1_and_MiscOrgans_startinput1,Results/f04_peaks/Liver_START2_and_Liver_startinput1,Results/f04_peaks/Brain_GROCap1_and_Brain_GRO1,...,max_Chr,max_Start,max_End,max_Stat,cho_file,cho_file_peak_name,cho_Chr,cho_Start,cho_End,cho_Stat
ID,,,,,,,,,,,,,,,,,,,,,
4,NW_020822601.1,37427816,37427999,+,129.031738,Results/f04_peaks/MiscOrgans_START1_and_MiscOr...,23,NW_020822601.1-18,NaN,NW_020822601.1-1445,...,NW_020822601.1,37427849,37427999,457.80,Results/f04_peaks/CHO_START1_and_CHO_startinput1,NW_020822601.1-41,NW_020822601.1,37427849,37427999,91.90
5,NW_020822619.1,2231719,2231926,-,16.203333,Results/f04_peaks/MiscOrgans_START1_and_MiscOr...,24,NW_020822619.1-12,NaN,NW_020822619.1-53,...,NW_020822619.1,2231771,2231921,61.30,,,,-1,-1,0.00
6,NW_020822577.1,6314085,6314339,+,11.444000,Results/f04_peaks/MiscOrgans_START1_and_MiscOr...,25,NW_020822577.1-68,NW_020822577.1-6,NaN,...,NW_020822577.1,6314179,6314329,77.80,Results/f04_peaks/CHO_START1_and_CHO_startinput1,NW_020822577.1-193,NW_020822577.1,6314096,6314246,6.12
8,NW_020822435.1,461540,462075,-,6.744815,Results/f04_peaks/MiscOrgans_START1_and_MiscOr...,27,NW_020822435.1-25,NaN,NW_020822435.1-42,...,NW_020822435.1,461768,461918,15.00,Results/f04_peaks/CHO_START1_and_CHO_startinput1,NW_020822435.1-23,NW_020822435.1,461773,461923,10.70
12,NW_020822644.1,1117287,1117499,-,3.882334,Results/f04_peaks/Liver_START2_and_Liver_start...,6,NaN,NW_020822644.1-33,NaN,...,NW_020822644.1,1117287,1117437,10.60,,,,-1,-1,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540101,NW_020822703.1,694062,694270,+,4.267500,Results/f04_peaks/Brain_GROCap1_and_Brain_GRO1...,8,NaN,NaN,NW_020822703.1-33,...,NW_020822703.1,694076,694226,8.85,,,,-1,-1,0.00
540110,NW_020822610.1,4671020,4671224,+,3.941875,Results/f04_peaks/MiscOrgans_START1_and_MiscOr...,16,NW_020822610.1-219,NW_020822610.1-161,NW_020822610.1-362,...,NW_020822610.1,4671050,4671200,8.00,Results/f04_peaks/CHO_START1_and_CHO_startinput1,NW_020822610.1-199,NW_020822610.1,4671020,4671170,4.73
540113,NW_020822415.1,54073641,54073947,+,5.232727,Results/f04_peaks/Brain_GROCap1_and_Brain_GRO1...,22,NaN,NaN,NW_020822415.1-1038,...,NW_020822415.1,54073755,54073905,20.80,Results/f04_peaks/CHO_GROCap1_and_CHO_GRO1,NW_020822415.1-210,NW_020822415.1,54073755,54073905,20.80
